
<div style="
    background-color: #f7f7f7;
    background-image: url(''), url('') ;
    background-position: left bottom, right top;
    background-repeat: no-repeat,  no-repeat;
    background-size: auto 60px, auto 160px;
    border-radius: 5px;
    box-shadow: 0px 3px 1px -2px rgba(0, 0, 0, 0.2), 0px 2px 2px 0px rgba(0, 0, 0, 0.14), 0px 1px 5px 0px rgba(0,0,0,.12);">

<h1 style="
    color: #2a4cdf;
    font-style: normal;
    font-size: 2.25rem;
    line-height: 1.4em;
    font-weight: 600;
    padding: 30px 200px 0px 30px;"> 
        Query NOMAD for perovskite compositions</h1>

<p style="
    line-height: 1.4em;
    padding: 30px 200px 0px 30px;">
        This notebook demonstrates how query NOMAD for perovskite compositions. The notebook also shows how to validate the ions against the <a href="https://github.com/Jesperkemist/Perovskite_composition/blob/main/perovskite_composition_schema.json" target="_blank">JSON Schema</a>

<p style="font-size: 1.25em; font-style: italic; padding: 5px 200px 30px 30px;">
    José A. Márquez and Hampus Näsström
</p>
</div>

### Basic imports

In [1]:
import json

import requests

base_url = 'http://nomad-lab.eu/prod/v1/api/v1'

### Query for a lead perovskite with band gap less than 1.65 eV

In [2]:
response = requests.post(
    f'{base_url}/entries/archive/query',
    json={
        'owner': 'visible',
        'query': {
            'results.eln.sections:any': ['PerovskiteComposition'],
            'results.material.elements:all': ['Pb'],
            'results.properties.electronic.band_gap.value': {
                'lte': 2.6435914460999996e-19
            },
        },
        'pagination': {'page_size': 1},
    },
)
response_json = response.json()
composition_data = response_json['data'][0]['archive']['data']
print(json.dumps(composition_data, indent=2))

{
  "m_def": "perovskite_solar_cell_database.composition.PerovskiteComposition",
  "short_form": "MAPbI",
  "long_form": "MAPbI3",
  "composition_estimate": "Estimated from precursor solutions",
  "sample_type": "Polycrystalline film",
  "dimensionality": "3D",
  "band_gap": 1.63,
  "name": "MAPI",
  "datetime": "2025-03-31T09:09:48.394259+00:00",
  "ions_a_site": [
    {
      "m_def": "perovskite_solar_cell_database.composition.PerovskiteAIonComponent",
      "system": "../uploads/ztDkTFJETdiMgrnRQSgsEA/archive/ZHsLIxapQ6idJZjA25Zohc8FRZzr#/data",
      "common_name": "Methylammonium",
      "molecular_formula": "CH6N+",
      "smiles": "C[NH3+]",
      "iupac_name": "methylazanium",
      "cas_number": "17000-00-9",
      "abbreviation": "MA",
      "source_compound_molecular_formula": "CH5N",
      "source_compound_smiles": "CN",
      "source_compound_iupac_name": "methanamine",
      "source_compound_cas_number": "74-89-5",
      "coefficient": "1"
    }
  ],
  "ions_b_site": [
 

### Validate response against JSON Schema

In [3]:
from jsonschema import ValidationError, validate

schema_id = (
    'https://raw.githubusercontent.com/Jesperkemist/'
    'Perovskite_composition/v1.0.0/perovskite_composition_schema.json'
)

# Load the JSON schema from the URL
schema_response = requests.get(schema_id)
schema = schema_response.json()

# Validate ion_data against the schema
try:
    validate(instance=composition_data, schema=schema)
    print(f'composition_data is valid against the {schema_id} schema.')
except ValidationError as e:
    print('Validation error:', e.message)

composition_data is valid against the https://raw.githubusercontent.com/Jesperkemist/Perovskite_composition/v1.0.0/perovskite_composition_schema.json schema.
